### Introduction

Humour, even for us humans, can be mysterious. It is no wonder that it poses a unique challenge for AI systems as well. Let us think about it - we all have that one friend who effortlessly brings laughter into our lives. They have a natural knack for timing, delivery, and a deep understanding of what makes something funny. But have we ever tried to explain why they are funny?

In this project, we aim to tackle the challenging task of creating an AI bot that excels in generating new jokes. This task is particularly difficult due to the complexities of humour. Humour is subjective and context-dependent, making it challenging for an AI system to understand and replicate effectively.

Additionally, jokes often rely on wordplay, sarcasm, and cultural references, which further complicates the task of generating original and funny jokes. Despite these challenges, we are determined to push the boundaries of AI and humour, striving to create a bot that can bring joy and laughter to users worldwide.

Meet **ChuckleChief**, our enthusiastic and curious novice AI companion, eager to unravel the mysteries of humour.

### Install and Import Dependencies

In [ ]:
!pip install accelerate -U better_profanity datasets transformers

In [ ]:
import nltk
nltk.download(["stopwords", "wordnet"])

In [ ]:
import json
import os
import random
import re

import better_profanity as bp
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import torch
from tqdm import tqdm
from transformers import (
    AdamW,
    AutoConfig,
    DataCollatorForLanguageModeling,
    EarlyStoppingCallback,
    IntervalStrategy,
    GPT2LMHeadModel,
    GPT2Tokenizer,
    Trainer,
    TrainingArguments
)

from datasets import Dataset

In [ ]:
from google.colab import drive
drive.mount("/content/drive/")

### Load the Data

| Dataset Name | Description | Source | Number of Jokes | Format |
| --- | --- | --- | --- | --- |
| Short Jokes | A collection of short jokes in English | Kaggle | 231,657 | CSV |
| Joke Dataset - Stupidstuff | A dataset of English plaintext jokes from stupidstuff.org | GitHub | 3,770 | JSON |
| Joke Dataset - Wocka | A dataset of English plaintext jokes from wocka.com | GitHub | 10,000 | JSON |

The Short Jokes dataset from Kaggle is a large collection of short jokes in English that includes both one-liners and longer jokes. The dataset contains 231,657 jokes in CSV format.

The Joke Dataset from taivop/joke-dataset on GitHub includes two separate datasets: Stupidstuff and Wocka. The Stupidstuff dataset contains 3,770 English plaintext jokes scraped from stupidstuff.org, while the Wocka dataset contains 10,000 English plaintext jokes scraped from wocka.com. Both datasets are in JSON format and contain additional fields such as category, title, and rating.

Therefore, the total number of jokes in all three datasets combined is 245,427 jokes.

In [ ]:
def load_joke_data():
    """Load joke data"""
    short_jokes_df = pd.read_csv("short_jokes.csv")

    with open("stupidstuff.json", "r") as file:
        stupidstuff = json.load(file)
    stupidstuff_df = pd.DataFrame(stupidstuff, columns=["id", "body", "category"])

    with open("wocka.json", "r") as file:
        wocka = json.load(file)
    wocka_df = pd.DataFrame(wocka, columns=["id", "title", "body", "category"])

    return short_jokes_df, stupidstuff_df, wocka_df


short_jokes_df, stupidstuff_df, wocka_df = load_joke_data()

In [ ]:
short_jokes_df.head(2)

In [ ]:
stupidstuff_df.head(2)

In [ ]:
wocka_df.head(2)

### Preprocess Data

- Remove special characters.
- Remove numbers.
- Strip leading/trailing whitespace.
- Remove newlines and carriage returns.
- Check if the joke is clean.

In [ ]:
def preprocess_joke(joke):
    """Preprocess jokes"""
    joke = re.sub(r"[^\w\s.,?!]", "", joke)
    joke = re.sub("\d", "", joke)
    joke = joke.strip()
    joke = joke.replace("\n", " ").replace("\r", "")

    return joke

In [ ]:
short_jokes_df["Joke"] = short_jokes_df["Joke"].apply(preprocess_joke)
stupidstuff_df["body"] = stupidstuff_df["body"].apply(preprocess_joke)
wocka_df["body"] = wocka_df["body"].apply(preprocess_joke)

In [ ]:
short_jokes_df.shape, stupidstuff_df.shape, wocka_df.shape

In [ ]:
all_jokes = (
    short_jokes_df["Joke"].tolist()
    + stupidstuff_df["body"].tolist()
    + wocka_df["body"].tolist()
)

In [ ]:
print(len(all_jokes))

In [ ]:
def sample_jokes(all_jokes, n):
    """Sample jokes"""
    joke_indexes = np.random.randint(0, len(all_jokes), n)
    sampled_jokes = [all_jokes[index] for index in joke_indexes]
    return sampled_jokes

sampled_15000_jokes = sample_jokes(all_jokes, 15000)

In [ ]:
def is_clean(joke):
    """Check if a joke is clean"""
    if bp.profanity.contains_profanity(joke):
        return False
    else:
        return True

In [ ]:
def filter_jokes(jokes):
    """Filter out offensive jokes"""
    clean_jokes = []
    for joke in tqdm(jokes):
        if is_clean(joke):
            clean_jokes.append(joke)
    return clean_jokes

clean_jokes = filter_jokes(sampled_15000_jokes)

In [ ]:
len(clean_jokes)

In [ ]:
clean_jokes_df = pd.DataFrame(clean_jokes, columns=["jokes"])
clean_jokes_df.to_csv("clean_jokes_2.csv", index=False)

In [ ]:
clean_jokes_df = pd.read_csv("clean_jokes_2.csv")

In [ ]:
def get_subset(jokes_df, n):
    """Get data subset"""
    clean_jokes = [
        str(joke).strip() for joke in jokes_df["jokes"] if len(str(joke).strip()) >= 10
    ]
    random_jokes = random.sample(clean_jokes, n)
    subset_df = pd.DataFrame(random_jokes, columns=["jokes"])
    return subset_df

In [ ]:
def split_data(jokes_df, train_size=0.8, val_size=0.1, test_size=0.1):
    """Split data"""
    total_size = len(jokes_df)

    # Calculate the number of examples for each set.
    train_num = int(train_size * total_size)
    val_num = int(val_size * total_size)
    test_num = int(test_size * total_size)

    all_indices = np.arange(total_size)
    train_indices = np.random.choice(all_indices, train_num, replace=False)
    val_indices = np.random.choice(
        np.setdiff1d(all_indices, train_indices), val_num, replace=False
    )
    test_indices = np.setdiff1d(
        all_indices, np.concatenate([train_indices, val_indices])
    )

    # Split data based on indices
    train_df = jokes_df.iloc[train_indices]
    val_df = jokes_df.iloc[val_indices]
    test_df = jokes_df.iloc[test_indices]

    return train_df, val_df, test_df

### Model Comparison

- `bert-base-uncased`: BERT (Bidirectional Encoder Representations from Transformers) is a widely-used transformer-based model that has achieved state-of-the-art performance on various natural language processing tasks. The "base" variant refers to its medium-sized configuration, offering a balance between model size and performance. "uncased" indicates that the model treats all text as lowercase, disregarding capitalization. BERT incorporates a deep bidirectional transformer encoder, capturing contextual information from both preceding and following words. It is pretrained on a large corpus and can be fine-tuned for specific tasks.

- `distilbert-base-uncased`: DistilBERT is a distilled version of BERT, striking a good balance between performance and efficiency. It retains competitive performance while being smaller and faster than the original BERT model. Like BERT, "uncased" signifies that the model operates with lowercase text. DistilBERT achieves efficiency gains through techniques such as knowledge distillation and parameter reduction. Its reduced size makes it more manageable and quicker to fine-tune, particularly in scenarios with limited computational resources or smaller training datasets.  

- `gpt-2`: GPT-2 (Generative Pre-trained Transformer 2) is a cutting-edge language model explicitly designed for text generation tasks. Renowned for its ability to produce high-quality and coherent text, GPT-2 is particularly well-suited for joke generation. Built upon a transformer architecture with a substantial number of parameters, GPT-2 captures long-range dependencies in input text effectively.

We will start with fine-tuning `gpt-2 (124M parameter)` model for joke generation.

### Load Model and Tokeniser

In [ ]:
model = GPT2LMHeadModel.from_pretrained("gpt2")
tokeniser = GPT2Tokenizer.from_pretrained("gpt2")

In [ ]:
print(f"The model has {sum(p.numel() for p in model.parameters()):,} trainable parameters")

### Add Custom Tokens

In [ ]:
bos = "<|endoftext|>"  # Beginning of sequence token
eos = "<|eos|>"        # End of sequence token
pad = "<|pad|>"        # Padding token

special_tokens = {"bos_token": bos, "eos_token": eos, "pad_token": pad}

# Add custom tokens to the tokeniser.
new_tokens = tokeniser.add_special_tokens(special_tokens)

# Model config with custom tokens.
config = AutoConfig.from_pretrained(
    "gpt2",
    bos_token_id=tokeniser.bos_token_id,
    eos_token_id=tokeniser.eos_token_id,
    pad_token_id=tokeniser.pad_token_id,
    output_hidden_states=False
)

# Load model with config.
model = GPT2LMHeadModel.from_pretrained("gpt2", config=config)

# Resize embeddings to include new tokens.
model.resize_token_embeddings(len(tokeniser))

### Define a Helper Function to Generate Jokes

In [ ]:
def generate_jokes(model, tokeniser, prompt, num_jokes=1, max_len=50):
    filtered_jokes = []

    while len(filtered_jokes) < num_jokes:
        input_ids = tokeniser.encode(prompt, return_tensors="pt")
        generated_text_samples = model.generate(
            input_ids,
            max_length=max_len,
            num_return_sequences=num_jokes,
            repetition_penalty=1.2,
            temperature=0.75,
            do_sample=True
        )

        generated_jokes = [
            tokeniser.decode(joke, skip_special_tokens=True)
            for joke in generated_text_samples
        ]

        # Apply filter to keep only inoffensive jokes.
        for joke in generated_jokes:
            if is_clean(joke):
                filtered_jokes.append(joke)

    return filtered_jokes

In [ ]:
clean_jokes_subset = get_subset(clean_jokes_df, 11000)
train_df, val_df, test_df = split_data(clean_jokes_subset)
train_df.shape, val_df.shape, test_df.shape

In [ ]:
train_jokes = Dataset.from_pandas(train_df[["jokes"]])
val_jokes = Dataset.from_pandas(val_df[["jokes"]])
test_jokes = Dataset.from_pandas(test_df[["jokes"]])

### Tokenise and Pad Data

In [ ]:
def tokenise_data(data, max_length=512):
    return data.map(
        lambda x: tokeniser(
            x["jokes"],
            padding="max_length",
            truncation=True,
            max_length=max_length
        ),
        batched=True,
    )

tokenised_train_jokes = tokenise_data(train_jokes)
tokenised_val_jokes = tokenise_data(val_jokes)
tokenised_test_jokes = tokenise_data(test_jokes)

### Set up the Training Arguments and Data Collator

In [ ]:
model_path = "./ChuckleChief"

training_args = TrainingArguments(
    output_dir=model_path,
    num_train_epochs=4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_dir=model_path,
    prediction_loss_only=True,
    evaluation_strategy=IntervalStrategy.STEPS,
    eval_steps=500,
    save_steps=2000,
    save_total_limit=2,
    load_best_model_at_end=True
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokeniser, mlm=False)

### Train the Model

In [ ]:
torch.cuda.empty_cache()

In [30]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenised_train_jokes,
    eval_dataset=tokenised_val_jokes,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=2)]
)
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss
500,6.303400,4.218644
1000,4.233000,4.113698
1500,4.008900,4.073078
2000,3.984900,4.045985
2500,3.855800,4.033286
3000,3.801600,4.024386
3500,3.764100,4.024865
4000,3.719600,4.019519


TrainOutput(global_step=4400, training_loss=4.164508556019176, metrics={'train_runtime': 5510.0658, 'train_samples_per_second': 6.388, 'train_steps_per_second': 0.799, 'total_flos': 9197479526400000.0, 'train_loss': 4.164508556019176, 'epoch': 4.0})

### Evaluate the Model

In [32]:
trainer.evaluate(tokenised_test_jokes)

{'eval_loss': 3.97096848487854,
 'eval_runtime': 53.2835,
 'eval_samples_per_second': 20.644,
 'eval_steps_per_second': 2.59,
 'epoch': 4.0}

### Save the Model

In [ ]:
new_model_path = os.path.join(model_path, "model")
if not os.path.exists(new_model_path):
    os.makedirs(new_model_path)

trainer.save_model(new_model_path)
tokeniser.save_pretrained(new_model_path)

### Generate Jokes

In [ ]:
loaded_model = GPT2LMHeadModel.from_pretrained(new_model_path)
loaded_tokeniser = GPT2Tokenizer.from_pretrained(new_model_path)

In [39]:
prompt = "Here is a joke filled with harmless humour:"
jokes = generate_jokes(loaded_model, loaded_tokeniser, prompt, num_jokes=3, max_len=30)
jokes

['Here is a joke filled with harmless humour: You know how it goes when you cross the road. If they hit your car, Im going to tell',
 'Here is a joke filled with harmless humour: A horse walks into the bar... And looks at him, his face just reddens up.  Which',
 'Here is a joke filled with harmless humour: When you talk about the Internet, how can I get around it? Unless they allow me to run away']

### Acknowledgement: Datasets

#### [Short Jokes](https://www.kaggle.com/datasets/abhinavmoudgil95/short-jokes)
#### [Stupidstuff](https://github.com/taivop/joke-dataset#stupidstuffjson)
#### [Wocka](https://github.com/taivop/joke-dataset#wockajson)